# Manila - Jeepney Route Connection and Genetic Algorithm - Modified CITY_GRAPH

## Route Connection and Genetic Algorithm Functions

### Setting Up

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import shapely
import folium
import geojson
import math
import osmnx as ox
from rtree import index as rtree_index
import pickle
import copy
import utm

from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from geopy.distance import geodesic
from shapely.ops import split

from __future__ import absolute_import, division
from math import radians, sin, cos, sqrt, atan2, exp, log
import webbrowser
import random
from scipy.spatial import KDTree
from scipy.spatial.distance import euclidean

ox.settings.log_console=True
ox.settings.use_cache=True

WALKING_DISTANCES = [300,550,800]
MAX_DISTANCE = 15
CONNECTION_TYPES = ["Default", "Area", "Degree", "Mixed"]

In [3]:
# Defining classes for the dataframes      
class stopCandidate:
    def __init__(self, lat, long, isTranspo, id, area):
        self.lat = lat
        self.long = long
        self.isTranspo = isTranspo
        self.enabled = False
        self.id = id #Stop ID
        self.area = area
        self.degree = 0
        
    def enable(self):
        self.enabled = True
        
    def disable(self):
        self.enabled = False
        
    def getLat(self):
        return self.lat
    
    def getLong(self):
        return self.long
    
    def getArea(self):
        return self.area
    
    def getDegree(self):
        return self.degree
    
class networkObj:
    def __init__(self, routes, stops, graph, conn_type, walk_distance):
        self.routes = routes
        self.stops = stops
        self.fitness_score = 0
        self.graph = graph
        self.conn_type = conn_type
        self.walk_distance = walk_distance
        
class Route:
    def __init__(self, route, route_id, length):
        self.route = route
        self.route_id = route_id
        self.length = length

In [4]:
# For units
def degrees_to_meters(angle_degrees):
    return angle_degrees * 6371000 * math.pi / 180

def meters_to_degrees(distance_meters):
    return distance_meters / 6371000 * 180 / math.pi

In [5]:
# Import and Export networks or graphs to pickle
def export_networks(networks, path):
    with open(path, 'wb') as f:
        pickle.dump(networks, f)

def import_networks(path):
    with open(path, 'rb') as f:
        routes = pickle.load(f)
    return routes

#### Graph and Features

In [6]:
# NOTE: SELECT THE CITY HERE, COMMENT OUT THE REMAINING CITIES
select_city = "Manila, Philippines"
city_file = 'map/Manila.graphml'


# GENERATION OF MAIN CITY GRAPH
# IF FIRST TIME RUNNING, RUN THIS CODE TO GENERATE THE GRAPH
def generate_graph():
    mode = 'drive'
    graph = ox.graph_from_place(select_city, network_type = mode) # Generate graph of Metro manila
    ox.save_graphml(graph, city_file) # Save it as a file

def load_graph():
    graph = ox.load_graphml(city_file)
    
    print("Graph loaded successfully")
    print("NUMBER OF EDGES: ", graph.number_of_edges())
    print("NUMBER OF NODES: ", graph.number_of_nodes())
    print('\n')
    return graph

# NOTE: Only run this if you do not have the graph
generate_graph()

# THIS IS THE MAIN GRAPH FOR THE CITY TO BE USED FOR ALL FUNCTIONS
CITY_GRAPH = load_graph()

Graph loaded successfully
NUMBER OF EDGES:  12617
NUMBER OF NODES:  4926




In [7]:
### For Filtering the roads and other features
# GETTING ROADS AND WATERWAYS

# Get all the roads in Manila
road = ox.graph_to_gdfs(CITY_GRAPH,nodes=False, edges=True)


# Get all the roads that are not junctions (ex. Roundabouts, intersection, etc.)
filtered_roads = road[road['junction'].isna()]

# Separate roads whose highway types are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

# Allowed Roads to place stops
filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']

# To separate zones
separation_options = ['primary', 'secondary', 'tertiary', 'unclassified']

# Get the roads whose widths are above the threshold
def check_list(lst):
    return any(x in filter_options for x in lst)

# Download OpenStreetMap data for the area of interest
waterways = ox.features_from_place(select_city, tags={'waterway': True})
filtered_rivers = waterways[waterways['waterway'].isin(['river'])]
filtered_streams = waterways[waterways['waterway'].isin(['stream'])]

# Get all the roads with the allowed road types
filtered_roads_strings = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 
filtered_roads_lists = rows_with_lists[rows_with_lists['highway'].apply(check_list)]

# Create spatial index
filtered_roads_strings_sindex = filtered_roads_strings.sindex
filtered_roads_lists_sindex = filtered_roads_lists.sindex
filtered_rivers_sindex = filtered_rivers.sindex
filtered_streams_sindex = filtered_streams.sindex

In [8]:
# This function will find which road or river intersects between amenities

def find_intersecting_features(line):
    # Check intersection with filtered roads
    possible_matches_roads = filtered_roads_strings.iloc[list(filtered_roads_strings_sindex.intersection(line.bounds))]
    for index, row in possible_matches_roads.iterrows():
        if line.intersects(row['geometry']) and row['highway'] in separation_options:
            return True

    possible_matches_lists = filtered_roads_lists.iloc[list(filtered_roads_lists_sindex.intersection(line.bounds))]
    for index, row in possible_matches_lists.iterrows():
        if line.intersects(row['geometry']):
            list_highway = row['highway']
            if any(x in separation_options for x in list_highway):
                return True
    
    # Check intersection with filtered rivers
    possible_matches_rivers = filtered_rivers.iloc[list(filtered_rivers_sindex.intersection(line.bounds))]
    for index, row in possible_matches_rivers.iterrows():
        if line.intersects(row['geometry']):
            return True

    # Check intersection with filtered streams
    possible_matches_streams = filtered_streams.iloc[list(filtered_streams_sindex.intersection(line.bounds))]
    for index, row in possible_matches_streams.iterrows():
        if line.intersects(row['geometry']):
            return True
    
    return False

### Stop Placement

In [36]:
# ADD POINTS TO NX GRAPH
# Function to add only points to the networkX graph
# The other functions focuses on adding polygons, this function just iterates and adds points

def add_points_to_graph(graph, graph_to_add):
    for node_key, node_data in graph.nodes.items():
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Point']:   
            graph_to_add.add_node(node_key, geometry=node_data['geometry'], name=node_data['name'], lat=node_data['lat'], amenity=node_data['amenity'],
                                lon=node_data['lon'])
                
                

In [37]:

# CREATING STOPS
# It should return a list of coordinates/nodes for stop and a graph of stops
# if residential area, check if the population density

# Global Variables used:
# list_of_stops - List of stops
# graph_of_stops - graph of all stops placed
import random


def place_stops_on_roads(amenity_graph, graph_of_stops, list_of_stops):
    list_relevant_edges_set = []
    string_relevant_edges_set = []
    
    stop_id = 0
    for node_key, node_data in amenity_graph.nodes(data=True):
        # All tranportation points are automatically stops
        if node_data['geometry'].geom_type in ['Point']:
            if node_data['amenity'] == 'transportation':
                # Find the nearest edge to the location point
                nearest_edge = ox.distance.nearest_edges(graph_of_stops, X=node_data['x'], Y=node_data['y'], return_dist=False)
                node1, node2, key = nearest_edge
                edge = graph_of_stops.get_edge_data(node1, node2)
                data = list(edge.values())[0]
            
                
                # get the line
                if 'geometry' not in data:
                    gdf_edges = ox.graph_to_gdfs(graph_of_stops, nodes=False, edges=True)
                    line = gdf_edges.loc[nearest_edge]['geometry']
                    edge_length = gdf_edges.loc[nearest_edge]['length']
                else:
                    line = data['geometry']
                    edge_length = data['length']
                    
                
                if edge_length > 20:
                    # x - lon - 120
                    # y - lat - 14
                    # LineString(lon, lat)
                    # geodesic(lat, lon)
                    
                    print(f"STOP ID: {stop_id}")
                    if node1 < 10000:
                        notice = '!'
                        print(f"SEEN ADDED NODE1{notice} {node1} - {graph_of_stops.nodes[node1]['y']}, {graph_of_stops.nodes[node1]['x']}")
                        
                    if node2 < 10000:
                        notice = '!'
                        print(f"SEEN ADDED NODE2{notice} {node2} - {graph_of_stops.nodes[node2]['y']}, {graph_of_stops.nodes[node2]['x']}")
                        
                    print(f"EDGE NODES ({node1}, {node2})")
                    
                    point_a = (graph_of_stops.nodes[node1]['y'], graph_of_stops.nodes[node1]['x'])
                    point_c = (graph_of_stops.nodes[node2]['y'], graph_of_stops.nodes[node2]['x']) 
                    
                    if len(line.coords) == 2:
                        print("Transpo Stop WAY 1")
                        # Get a random position between the line
                        random_position = random.uniform(0.3 * edge_length, edge_length * 0.7)
                        
                        # Convert to UTM
                        node1_utm = utm.from_latlon(point_a[0], point_a[1])
                        node2_utm = utm.from_latlon(point_c[0], point_c[1])

                        utm_line = LineString([(node1_utm[0], node1_utm[1]), (node2_utm[0], node2_utm[1])])

                        # Calculate the coordinate along the edge at the random position
                        point_on_road_utm = calculate_coordinate_along_edge(utm_line, random_position)
                        to_convert = (point_on_road_utm[0], point_on_road_utm[1], node1_utm[2], node1_utm[3])
                        point_on_road = utm.to_latlon(*to_convert)
                        lon = point_on_road[1]
                        lat = point_on_road[0]
                    else:
                        print("Transpo Stop WAY 2")
                        # Accessing all coordinates
                        coordinates = list(line.coords)
                        coordinates = coordinates[1:-1]
                        # Randomly pick one coordinate
                        point_on_road = random.choice(coordinates)
                        lon = point_on_road[0]
                        lat = point_on_road[1]
                    
                    isTranspo = True
                
                    graph_of_stops.add_node(stop_id, x=lon, y=lat, isTranspo=isTranspo)
                    list_of_stops.append(stopCandidate(lat, lon, True, stop_id, 0))
                
                    # Get the points
                    point_b = (lat, lon) # y,x
                    
                    # Calculate new geometries
                    if len(line.coords) == 2:
                        line_ab = LineString([(point_a[1], point_a[0]), (point_b[1], point_b[0])])
                        line_bc = LineString([(point_b[1], point_b[0]), (point_c[1], point_c[0])])
                    else:
                        result = split(line, Point(point_b[1], point_b[0]))
                        new_lines = [geom for geom in result.geoms]
                        line_ab = new_lines[0]
                        line_bc = new_lines[1]
                    
                    print("POINT B: ", point_b)
                    
                    print("NEW LINES")
                    print(line_ab)
                    print(line_bc)
                    print(f"ORIGINAL LINE: {line}")
                    print("---------")
                    
                    # Calculate the new distances
                    distance_ab = geodesic(point_a, point_b).meters
                    distance_bc = geodesic(point_b, point_c).meters
                    
                    print("NEW DISTANCES")
                    print(distance_ab)
                    print(distance_bc)
                    print(f"ORIGINAL DISTANCE: {data['length']}")
                    print("---------")
                    
                    
                    # Get the edge data and adjust distances
                    edge_data = data.copy()
                    edge_data['length'] = distance_ab
                    edge_data['geometry'] = line_ab
                    graph_of_stops.add_edge(node1, stop_id, **edge_data)
                    
                    edge_data['length'] = distance_bc
                    edge_data['geometry'] = line_bc
                    graph_of_stops.add_edge(stop_id, node2, **edge_data)
                    
                    print(f"HAS EDGE BETWEEN {node1} and {stop_id} : {graph_of_stops.has_edge(node1, stop_id)}")
                    print(f"HAS EDGE BETWEEN {stop_id} and {node2} : {graph_of_stops.has_edge(stop_id, node2)}")

                    # Remove the original edge
                    graph_of_stops.remove_edge(node1, node2)
                    stop_id += 1
                    print()
                
                
        elif node_data['geometry'].geom_type in ['MultiPolygon', 'Polygon']:
            # Get the roads surrounding and inside the node polygons
            buffer_poly = node_data['geometry'].buffer(meters_to_degrees(30))
            relevant_edges = get_relevant_edges(buffer_poly, list_relevant_edges_set, string_relevant_edges_set)
            
            # Calculate the number of stops based on node size and population density
            node_size = degrees_to_meters(node_data['geometry'].area)
            
            # Place stops randomly on these roads
            stop_id = place_stops_along_edges(relevant_edges, buffer_poly, node_size, graph_of_stops, list_of_stops, stop_id)
            

def get_relevant_edges(polygon, list_relevant_edges_set, string_relevant_edges_set):
    relevant_edges = []
    
    # Check intersection with filtered roads
    possible_matches_roads = filtered_roads_strings.iloc[list(filtered_roads_strings_sindex.intersection(polygon.bounds))]
    for index, row in possible_matches_roads.iterrows():
        if polygon.intersects(row['geometry']) and row['highway'] in filter_options:
            row_name = row['name']
            if row_name not in string_relevant_edges_set:
                relevant_edges.append([index, row])
                string_relevant_edges_set.append(row_name)


    possible_matches_lists = filtered_roads_lists.iloc[list(filtered_roads_lists_sindex.intersection(polygon.bounds))]
    for index, row in possible_matches_lists.iterrows():
        if polygon.intersects(row['geometry']):
            list_highway = row['highway']
            if any(x in filter_options for x in list_highway):
                row_name = row['name']
                if row_name not in list_relevant_edges_set:
                    relevant_edges.append([index, row])
                    list_relevant_edges_set.append(row_name)
                
    return relevant_edges

def place_stops_along_edges(edges, polygon, node_size, graph_of_stops, list_of_stops, stop_id):
    # Place stops randomly along the edges within the polygon
    
    if len(edges) > 0:
        for edge in edges:
            # Getting the edge data
            index = edge[0]
            data = edge[1]
            node1 = index[0]
            node2 = index[1]
            
            
            # get the line
            if 'geometry' not in data:
                gdf_edges = ox.graph_to_gdfs(graph_of_stops, nodes=False, edges=True)
                line = gdf_edges.loc[(node1, node2, 0)]['geometry']
                edge_length = gdf_edges.loc[(node1, node2, 0)]['length']
            else:
                line = data['geometry']
                edge_length = data['length'] 
            
            if edge_length > 20:
                # x - lon - 120
                # y - lat - 14
                # LineString(lon, lat)
                # geodesic(lat, lon)
                print(f"Stop ID: {stop_id}")
                
                if node1 < 10000:
                        notice = '!'
                        print(f"SEEN ADDED NODE1{notice} ({node1}, {node2}) - {graph_of_stops.nodes[node1]['y']}, {graph_of_stops.nodes[node1]['x']}")
                        
                if node2 < 10000:
                    notice = '!'
                    print(f"SEEN ADDED NODE2{notice} ({node1}, {node2}) - {graph_of_stops.nodes[node2]['y']}, {graph_of_stops.nodes[node2]['x']}")

                
                # Get the coordinates of the two nodes
                point_a = (graph_of_stops.nodes[node1]['y'], graph_of_stops.nodes[node1]['x'])
                point_c = (graph_of_stops.nodes[node2]['y'], graph_of_stops.nodes[node2]['x'])
                
                if len(line.coords) == 2:
                    print("Along edges WAY 1")
                    # Get a random position between the line
                    random_position = random.uniform(0.3 * edge_length, edge_length * 0.7)
                    
                    # Convert to UTM
                    node1_utm = utm.from_latlon(point_a[0], point_a[1])
                    node2_utm = utm.from_latlon(point_c[0], point_c[1])

                    utm_line = LineString([(node1_utm[0], node1_utm[1]), (node2_utm[0], node2_utm[1])])

                    # Calculate the coordinate along the edge at the random position
                    point_on_road_utm = calculate_coordinate_along_edge(utm_line, random_position)
                    to_convert = (point_on_road_utm[0], point_on_road_utm[1], node1_utm[2], node1_utm[3])
                    point_on_road = utm.to_latlon(*to_convert)
                    lon = point_on_road[1]
                    lat = point_on_road[0]
                else:
                    print("Along edges WAY 2")
                    # Accessing all coordinates
                    coordinates = list(line.coords)
                    coordinates = coordinates[1:-1]
                    # Randomly pick one coordinate
                    point_on_road = random.choice(coordinates)
                    lon = point_on_road[0]
                    lat = point_on_road[1]
                
                # Add the stop to graph and list
                isTranspo = False
                
                graph_of_stops.add_node(stop_id, x=lon, y=lat, isTranspo=isTranspo)
                list_of_stops.append(stopCandidate(lat, lon, False, stop_id, node_size))
                
                # Get the points
                point_b = (lat, lon)
                
                # Calculate new geometries
                
                if len(line.coords) == 2:
                    line_ab = LineString([(point_a[1], point_a[0]), (point_b[1], point_b[0])])
                    line_bc = LineString([(point_b[1], point_b[0]), (point_c[1], point_c[0])])
                else:
                    result = split(line, Point(point_b[1], point_b[0]))
                    new_lines = [geom for geom in result.geoms]
                    line_ab = new_lines[0]
                    line_bc = new_lines[1]
                    
                print("POINT B: ", point_b)
                
                print("NEW LINES")
                print(line_ab)
                print(line_bc)
                print("---------")
                
                # Calculate the new distances
                distance_ab = geodesic(point_a, point_b).meters
                distance_bc = geodesic(point_b, point_c).meters
                
                print("NEW DISTANCES")
                print(distance_ab)
                print(distance_bc)
                print(f"ORIGINAL DISTANCE: {data['length']}")
                print("---------")

                # Add the new distances and new geometries
                edge_data = data.copy()
                edge_data['length'] = distance_ab
                edge_data['geometry'] = line_ab
                graph_of_stops.add_edge(node1, stop_id, **edge_data)
                
                edge_data['length'] = distance_bc
                edge_data['geometry'] = line_bc
                graph_of_stops.add_edge(stop_id, node2, **edge_data)
                
                print(f"HAS EDGE BETWEEN {node1} and {stop_id} : {graph_of_stops.has_edge(node1, stop_id)}")
                print(f"HAS EDGE BETWEEN {stop_id} and {node2} : {graph_of_stops.has_edge(stop_id, node2)}")

                # Remove the original edge
                graph_of_stops.remove_edge(node1, node2)
                stop_id += 1
                print()
                    
    return stop_id
        

def calculate_coordinate_along_edge(edge, position):
    # Calculate the coordinate along the edge at the given position
    point = edge.interpolate(position)
    return point.x, point.y
    

### Route Network Generation

In [38]:
# V2 - Graph with the snapping function
# Generate Route Network from connected routes

# Global Variables used:
# graph_of_stops - Graph of stops that will be used to create routes
def generate_route_network(stop_nodes, max_walking_dist, max_stops, max_routes, graph_of_stops, city_area_sum, connection_type="Default"):
    overall_graph = nx.MultiDiGraph() # The route network graph
    next_nodes = [n for n in stop_nodes]
    enable_stop_nodes(next_nodes)
    route_network = []
    num_routes = 0 # Count number of routes

    while num_routes < max_routes:
        route_id = f'{num_routes}-A' # This will be used as a key for the edge
        next_nodes = [n for n in stop_nodes] # Resets the list of nodes so that nodes can be reused in a different
        selected_node = random.choice(next_nodes) # For the first node
        next_nodes.remove(selected_node)
        route_gen, route_length = generate_route(selected_node, next_nodes, max_walking_dist, connection_type, max_stops, overall_graph, city_area_sum, route_id, graph_of_stops)
        
        if len(route_gen) > 1:  
            new_route = Route(route_gen, route_id, route_length) # Create a new route object to store the Route ID and the route itself
            
            add_to_graph(route_gen, overall_graph, graph_of_stops, route_id)
            
            route_network.append(new_route)   
            num_routes += 1
               
    return route_network, overall_graph

def add_to_graph(route, overall_graph, graph_of_stops, route_id):
    
    # Directly add nodes based on node identifiers
    for connection in route:
        overall_graph.add_node(connection[0], **graph_of_stops.nodes[connection[0]]) # The origin
        overall_graph.add_node(connection[-1], **graph_of_stops.nodes[connection[-1]]) # The destination
        
        distance_travelled = 0
        # Get the total distance from point A to point B
        for i in range(len(connection)-1):
            node_data = graph_of_stops.nodes[connection[i]]
            next_node_data = graph_of_stops.nodes[connection[i+1]]
            distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
        
        # Finally, Add the edge
        overall_graph.add_edge(connection[0], connection[-1], key=route_id, road_path = connection, distance = distance_travelled) # Add edge
        

# Generate route from stop nodes
def generate_route(source, next_nodes, max_walking_dist, connection_type, max_stops, network_graph, city_area_sum, route_id, graph_of_stops):
    short_route_list = [] # List of nx.shortest_path results
    totalDistance = 0
    orig_node = source
    num_stops = 0 # Count number of stops
    
    # CONFIGURATION
    max_tries = 3 # This is the max number of tries before breaking the loop || To avoid longer runtimes
    current_tries = 0

    while totalDistance < MAX_DISTANCE and num_stops < max_stops:
        
        #print(f"Selected node is {selected_node.getLat()}, {selected_node.getLong()}")
        enable_surrounding_nodes(next_nodes)
        disable_surrounding_nodes(next_nodes, orig_node, max_walking_dist)
        enabled_nodes = [n for n in next_nodes if n.enabled]
        if len(enabled_nodes) == 0:
            break
        
        #print(f"{len(enabled_nodes)} nodes out of {len(next_nodes)}")
        dest_node = get_enabled_node_with_highest_edge_probability(orig_node, enabled_nodes, connection_type, city_area_sum, network_graph) # Getting the destination node
        
        if (dest_node == None or dest_node.id == orig_node.id):
            break
        
        # Remove it as a candidate
        next_nodes.remove(dest_node)
        
        # This is to check if there is already an exiting edge in the route. If true, then it should not connect
        connection_edge1 = network_graph.has_edge(orig_node.id, dest_node.id, route_id)
        connection_edge2 = network_graph.has_edge(dest_node.id, orig_node.id, route_id)
        
        # If there is no possible path or there is atleast one existing edge, do not connect
        if not nx.has_path(graph_of_stops, orig_node.id, dest_node.id) or connection_edge1 or connection_edge2:
            current_tries += 1
            if current_tries == max_tries:
                break
        else:
            shortest_route = nx.shortest_path(graph_of_stops, orig_node.id, dest_node.id, weight='length')
            distance_travelled = 0
            # Get the total distance from point A to point B
            for i in range(len(shortest_route)-1):
                node_data = graph_of_stops.nodes[shortest_route[i]]
                next_node_data = graph_of_stops.nodes[shortest_route[i+1]]
                
                distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])

            # Checks if it does not exceed the max distance
            if totalDistance + distance_travelled <= MAX_DISTANCE:
                
                # Updating local degree count used for connection probability
                orig_node.degree += 1
                dest_node.degree += 1
                
                totalDistance += distance_travelled
                short_route_list.append(shortest_route)
                num_stops += 1
                
                orig_node = dest_node # Now change the origin to the destination
            else:
                break
    if len(short_route_list) > 4 and totalDistance > 7:
        print(f"# OF CONNECTIONS AND TOTAL DISTANCE: {len(short_route_list)} - {totalDistance}")
        return short_route_list, totalDistance
    
    else:
        return []

# Disable surrounding nodes
def disable_surrounding_nodes(next_nodes, source_node, max_distance):
    max_radius = 2000 # This is max radius in which all nodes outside will be disabled
    source = (source_node.getLat(), source_node.getLong())
    
    for node in next_nodes:
        point = (node.getLat(), node.getLong())
        distance_to_source = geodesic(source, point).meters
        if distance_to_source <= max_distance or distance_to_source > max_radius:
            node.disable()
            
# Enable surrounding nodes
def enable_surrounding_nodes(next_nodes):
    for node in next_nodes:
        node.enable()
        
def get_enabled_node_with_highest_edge_probability(source_node, enabled_nodes, connection_type, city_area_sum, network_graph):

    prob_list = []
    for n in enabled_nodes:
        edge_prob = get_edge_probability(source_node, n, len(enabled_nodes), connection_type, city_area_sum, network_graph)
        prob_list.append(edge_prob)
    
    
    min_score = min(prob_list)
    if min_score < 0: # Shift the scores to ensure all are positive
        prob_list = [score - min_score for score in prob_list]
    total = sum(prob_list)
    selection_p = [score / total for score in prob_list]
    
    chosen_node = np.random.choice(enabled_nodes, 1, p=selection_p)[0]    

    return chosen_node

# Probabilities of candidate nodes based on distance, area, node degree, and if transpo stop
def get_edge_probability(source, destination, normalization_factor, connection_type, city_area_sum, network_graph):
    source_coord = [source.getLat(), source.getLong()]
    dest_coord = [destination.getLat(), destination.getLong()]

    base_prob = exp(-(euclidean(source_coord, dest_coord))) / float(normalization_factor)
    
    # If there is already an existing edge between source and destination, decrease the probability
    if network_graph.has_edge(source.id, destination.id):
        base_prob *= 0.5 # The penalty

    if connection_type == "Default":
        if destination.isTranspo:
            return base_prob * 1.5
        return base_prob
    elif connection_type == "Area":
        if destination.isTranspo:
            return base_prob * 1.5
        return base_prob * (1 + (destination.getArea() / city_area_sum))
    elif connection_type == "Degree":
        if destination.isTranspo:
            return base_prob * 1.5 * (1 + (destination.getDegree() / 10))
        return base_prob * (1 + (destination.getDegree() / 10))

def radius(stops):
    circles = []
    for stop in stops:
        stop_point = Point(stop[1], stop[0])  # Create a Point object from [lat, lon] coordinates
        circle = stop_point.buffer(radius / 111000)  # Buffer the Point to create a circle (assuming 1 degree is approximately 111000 meters)
        circles.append(circle)
    return circles

def enable_stop_nodes(stop_nodes):
    for n in stop_nodes:
        n.enable()

def all_nodes_disabled(stop_nodes):
    return get_num_disabled(stop_nodes) == len(stop_nodes)

def get_num_disabled(stop_nodes):
    return sum(1 for n in stop_nodes if not n.enabled)

def haversine(lat1, lon1, lat2, lon2):
    # Use geopy's geodesic function to calculate the distance
    distance = geodesic((lat1, lon1), (lat2, lon2)).kilometers
    return distance

# Markers for visualization purposes
def add_markers(used_stops, network_graph):
    for stop in used_stops:
        #popup_text = f"Name: {stop.name}<br>Type: {stop.a_type}<br>Coordinates: {stop.getLat()}, {stop.getLong()}"
        lat = network_graph.nodes[stop]['y']
        long = network_graph.nodes[stop]['x']
        folium.Marker(location=[lat, long]).add_to(m)
        
def add_stops_to_list(routes):
    used_stops = []
    for route in routes:
        for conn in route.route:
            if conn[0] not in used_stops:
                used_stops.append(conn[0])
            if conn[-1] not in used_stops:
                used_stops.append(conn[-1])
    return used_stops

In [ ]:
# Method 1
# Iterate through each node in reverse and get the shortest path

def get_reverse_route(route, graph_of_stops, overall_graph):
    paths = route.route
    reverse_paths = paths[::-1]
    
    for connection in reverse_paths:
        reverse_connection = connection[::-1]
        origin_node = None
        
        for stop in reverse_connection:
            if origin_node == None:
                origin_node = stop
                continue
            
            # If its a one way
            # edge = graph_of_stops.edge(stop, origin_node)
            # if edge['oneway']:
            
            if nx.has_path(graph_of_stops, origin_node, stop):
                reverse_path = nx.shortest_path(graph_of_stops, origin_node, stop, weight='length')
                
                for i in range(len(reverse_path)-1):
                    node_data = graph_of_stops.nodes[reverse_path[i]]
                    next_node_data = graph_of_stops.nodes[reverse_path[i+1]]
                    distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
                
                
                route_id = route.route_id[:-1] + 'B'
                reverse_route_obj = Route(reverse_path, route_id, distance_travelled)
                overall_graph.add_edge(origin_node, stop, key=route_id, road_path = reverse_path, distance = distance_travelled) # Add edge
                
        return reverse_route_obj
    

In [ ]:
# # Method 2
# # Get the shortest path between the last stop and the first stop in the entire route itself

# def get_reverse_route(start_node, end_node, original_route_id, graph_of_stops, overall_graph, max_walking_distance):
    
#     if nx.has_path(graph_of_stops, start_node, end_node):
#         entire_reverse_route = nx.shortest_path(graph_of_stops, start_node, end_node, weight='length')
        
#         new_route_list = []
#         reverse_connection = []
#         for stop in entire_reverse_route:
#             # FIRST RUN If it is empty, append the first stop and continue
#             if len(reverse_connection) == 0:
#                 reverse_connection.append(stop)
#                 continue
            
#             # If it is not empty but the the route goes through a stop, append the stop
#             first_stop_in_list = reverse_connection[0]
#             source = (graph_of_stops.nodes[first_stop_in_list]['y'], graph_of_stops.nodes[first_stop_in_list]['x'])
#             destination = (graph_of_stops.nodes[stop]['y'], graph_of_stops.nodes[stop]['x'])
#             distance_to_source = geodesic(source, destination).meters
            
#             if stop < 100000 and distance_to_source > max_walking_distance:
#                 reverse_connection.append(stop)
#                 new_route_list.append(reverse_connection)
#                 reverse_connection = []
#                 reverse_connection.append(stop)
#                 continue
                
#             reverse_connection.append(stop)
            
#         print(new_route_list)
        
#         # Reverse Route ID
#         reverse_route_id = original_route_id[:-1] + 'B'
        
#         # Snap it on the graph
#         totalDistance = 0
#         for connection in new_route_list:
#             if not overall_graph.has_node(connection[0]):
#                 overall_graph.add_node(connection[0], **graph_of_stops.nodes[connection[0]]) # The origin
                
#             if not overall_graph.has_node(connection[-1]):
#                 overall_graph.add_node(connection[-1], **graph_of_stops.nodes[connection[-1]]) # The destination
            
#             distance_travelled = 0
#             # Get the total distance from point A to point B
#             for i in range(len(connection)-1):
#                 node_data = graph_of_stops.nodes[connection[i]]
#                 next_node_data = graph_of_stops.nodes[connection[i+1]]
#                 distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
            
#             # Finally, Add the edge
#             overall_graph.add_edge(connection[0], connection[-1], key=reverse_route_id, road_path = connection, distance = distance_travelled) # Add edge
#             totalDistance += distance_travelled
        
#         # Make the object
#         print(f"TOTAL REVERSE DISTANCE: {totalDistance}")
#         if totalDistance > MAX_DISTANCE:
#             print("ABOVE MAX DISTANCE")
#         return Route(new_route_list, reverse_route_id, totalDistance)
        
#     else:
#         return None

In [ ]:
# #TODO: CONNECT THE ROUTE REVERSAL
# # Reverse route traversal
# def get_reverse_route(network, graph_of_stops):
#     reverse_route_network = []
    
#     for route in network.routes:
#         index = len(route)-1
        
#         reverse_route = []
#         totalDistance = 0
#         while index >= 0:
#             connection = route[index] # Get the connection
#             rev_origin = connection[-1]
#             rev_dest = connection[0]
            
#             if nx.has_path(graph_of_stops, rev_origin, rev_dest):
#                 rev_path = nx.shortest_path(graph_of_stops, rev_origin, rev_dest, weight='length') # Get the path
                
#                 distance_travelled = 0
#                 # Get the total distance from point A to point B
#                 for i in range(len(rev_path)-1):
#                     node_data = graph_of_stops.nodes[rev_path[i]]
#                     next_node_data = graph_of_stops.nodes[rev_path[i+1]]
#                     distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
                    
#                 totalDistance += distance_travelled
                
#                 reverse_route.append(rev_path)
#                 index -= 1
#             else:
#                 # there is no reverse route for this so append nothing to the 
#                 reverse_route_network.append([])
#                 break
    
#         # Checks if it does not exceed the max distance
#         if totalDistance <= MAX_DISTANCE:
#             reverse_route_network.append(reverse_route)
#         else:
#             reverse_route_network.append([])
            
#     return reverse_route_network

In [9]:
for i, node in CITY_GRAPH.nodes(data=True):
    if i < 100000:
        print(i)

In [72]:
map_center = (14.599512, 120.984222)
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Iterate through each road
for index, road in filtered_roads_strings.iterrows():
    
    if road['name'] != 'Taft Avenue':
        continue
    line_coords = list(road['geometry'].coords)
    folium.PolyLine(locations=[(y, x) for x, y in line_coords], popup=f'{road['osmid']}', color='blue').add_to(m)

        
m.save('taft.html')

In [75]:
map_center = (14.599512, 120.984222)
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Iterate through each road
for index, road in filtered_roads_strings.iterrows():
    
    if road['highway'] != 'secondary':
        continue
    
    line_coords = list(road['geometry'].coords)
    folium.PolyLine(locations=[(y, x) for x, y in line_coords], popup=f'{road['osmid']}', color='blue').add_to(m)

        
m.save('taft.html')

In [71]:
for i, data in road.iterrows():
    if data['name'] == 'Taft Avenue':
        print(data)

osmid                                              1122524649
oneway                                                   True
lanes                                                       4
name                                              Taft Avenue
highway                                               primary
maxspeed                                                   60
reversed                                                False
length                                                 12.455
ref                                                       170
geometry    LINESTRING (120.9841469 14.5834839, 120.984090...
width                                                     NaN
access                                                    NaN
bridge                                                    NaN
junction                                                  NaN
tunnel                                                    NaN
Name: (12067373, 4223876367, 0), dtype: object
osmid                  

### Graph Error Checks

In [40]:
# # ERROR CHECK - Checking if there are duplicate routes in the list of routes
def check_duplicate_routes(network):
    routes = network.routes
    seen_routes = set()
    error = "xxx"

    for route_obj in routes:
        standardized_route = tuple(tuple(sorted(sublist)) for sublist in route_obj.route)

        if standardized_route in seen_routes:
            print(f"----- ERROR{error} FOUND EXACT DUPLICATE ROUTES")
        else:
            seen_routes.add(standardized_route)

In [41]:
# ERROR Check - Checks if routes is consistent with its stops
def check_stops_routes(network):
    error = "xxx"
    # Checks if each node in the route is in the list of stops
    for route in network.routes:
        for connection in route.route:
            if connection[0] not in network.stops:
                print(f"---- ERROR{error} MISSING ROUTE ORIGIN STOP IN LIST OF STOPS ", connection[0])
            if connection[-1] not in network.stops:
                print(f"---- ERROR{error} MISSING ROUTE DEST STOP IN LIST OF STOPS ", connection[-1])

In [42]:
# ERROR Check - Checks if the routes are in correct order
def check_order_route(routes):
    error = "xxx"
    for route in routes:
        for connection in route.route:
            if route.route.index(connection) > 0:
                if connection[0] != prev_connection[-1]:
                    print(f"---- ERROR{error} WRONG ORDER DETECTED --")
                    print(prev_connection[0], " - ", prev_connection[-1])
                    print(connection[0], " - ", connection[-1])
                    print("--------------------------")
            prev_connection = connection
        

In [43]:
# ERROR CHECK - Checks the consistency of the network with its routes
def check_graph_with_route(_network):
    error = "xxx"
    for route in _network.routes:
        for connection in route.route:
            if not _network.graph.has_node(connection[0]):
                print(f"---- ERROR{error} MISSING NODE IN GRAPH: ", connection[0])
            if not _network.graph.has_node(connection[-1]):
                print(f"---- ERROR{error} MISSING NODE IN GRAPH: ", connection[-1])
            if not _network.graph.has_edge(connection[0], connection[-1], route.route_id):
                print(f"---- ERROR{error} MISSING EDGE IN GRAPH: ", connection[0], " - ", connection[-1])
                
            if _network.graph.get_edge_data(connection[0], connection[-1], route.route_id) == None:
                print(f"---- ERROR{error} MISSING EDGE INFORMATION: ", connection[0], " - ", connection[-1])

In [44]:
# ERROR CHECK - Checks the consistency of the network with its list of stops
def check_graph_with_stops(_network):
    error = "xxx"
    # Checks if all stops in the list is in the graph
    for stop in _network.stops:
        if not _network.graph.has_node(stop):
            print(f"---- ERROR{error} MISSING LIST STOP IN GRAPH: ", stop)
            
    # Checks if all nodes in the graph are in the list
    for node, node_data in _network.graph.nodes(data=True):
        if node < 10000:
            if node not in _network.stops:
                print(f"---- ERROR{error} MISSING GRAPH NODE IN LIST: ", node)

### Simplicity

In [45]:
# TODO: WORKING IN PROGRESS
def simplicity_metric(network):
    routes = network.routes
    
    for route in routes:
        for i in range(len(route) - 1):
            u, v = route[i], route[i + 1]
            
            if graph_of_stops.has_edge(u, v):
                edge_data = graph_of_stops.get_edge_data(u, v)
            else:
                # Skip if there's no direct edge between u and v
                continue
            
            # Edge data might have multiple edges with different keys
            for key in edge_data:
                road_name = edge_data[key].get('name', 'Unnamed Road')
            
            
            
            # Compare

### Network Analysis Metrics

In [46]:
# Longest Route
def get_longest_route(network_routes):
    sorted_routes = sorted(network_routes, key=lambda x: x.length, reverse=True)
    longest_route = sorted_routes[0]
    
    stop_list = [] # Get all the stops
    for connection in longest_route.route:
        if connection[0] not in stop_list:
            stop_list.append(connection[0])
        if connection[-1] not in stop_list:
            stop_list.append(connection[-1])
    
    return longest_route, stop_list

# Shortest Route
def get_shortest_route(network_routes):
    sorted_routes = sorted(network_routes, key=lambda x: x.length, reverse=True)
    shortest_route = sorted_routes[-1]
    
    stop_list = [] # Get all the stops
    for connection in shortest_route.route:
        if connection[0] not in stop_list:
            stop_list.append(connection[0])
        if connection[-1] not in stop_list:
            stop_list.append(connection[-1])
    
    return shortest_route, stop_list

# Average Route Length
def get_average_route_length(network_routes):
    sum_length = sum([n.length for n in network_routes])
    average_length = sum_length / len(network_routes)
    return average_length

# Network Diameter
def get_network_diameter(network_graph):
    diameter = nx.diameter(network_graph)
    return diameter

### Visualizations

In [47]:
# Function to plot/visualize connected zones on the map
import random

# This is to better visualize the networks
def plot_connected_zones_network_on_map(graph, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    colors = [
    "Red", "Green", "Blue", "Yellow", "Orange", "Purple", "Cyan", "Magenta", "Maroon",
    "Olive", "Lime", "Teal", "Navy", "Aqua", "Fuchsia", "Coral", "Indigo", "Violet"]
    
    color_map = {}

    # Iterate over the nodes in the network
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                
                network_id = data["network_id"]
                
                if network_id not in color_map:
                    color = random.choice(colors)
                    color_map[network_id] = color
                else:
                    color = color_map[network_id]
                
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m


In [48]:
# This is to visualize the stops
def plot_stops_on_map(stops):
    # Iterate over the nodes in the network
    map_center = (14.599512, 120.984222)
    network_map = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
    for stop in stops:
        
        if stop.isTranspo:
            marker_color = 'blue'
        else:
            marker_color = 'red'
            
        folium.Marker(location=[stop.lat, stop.long], popup=f"Transportation: {stop.isTranspo}", icon=folium.Icon(color=marker_color)).add_to(network_map)
        
    return network_map

In [73]:
#Plotting filtered roads FOR VISUALIZATION ONLY
def plot_all_filtered_roads():
    map_center = (14.599512, 120.984222)
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in filtered_roads_strings.iterrows():
        line_coords = list(road['geometry'].coords)

        if road['highway'] == 'primary':
            folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

    #     if road['highway'] == 'secondary':
    #         folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='red').add_to(m)

    #     if road['highway'] == 'tertiary':
    #         folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='green').add_to(m)
            
    #     if road['highway'] == 'trunk':
    #         folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='black').add_to(m)
            
    #     if road['highway'] == 'residential':
    #         folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='brown').add_to(m)

    #     if road['highway'] == 'unclassified':
    #         folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='orange').add_to(m)

    # for index, road in filtered_roads_lists.iterrows():
    #     line_coords = list(road['geometry'].coords)
    #     folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='purple').add_to(m)

    # Return the map
    return m

### Network Coverage

In [50]:
# Node Centrality
def get_node_centrality(network_graph):
    degree_centrality = nx.degree_centrality(network_graph)
    betweenness_centrality = nx.betweenness_centrality(network_graph)
    closeness_centrality = nx.closeness_centrality(network_graph)
    
    return degree_centrality, betweenness_centrality, closeness_centrality

In [51]:
# Grid Ratio

## Main

### Manila Test

#### Reading Data

In [52]:
# File Paths
Manila_pikl_filepath = "Saved Networks/Manila/"
Manila_map_filepath = "Saved Maps/Manila/"

In [53]:
# READING DATA (All Amenities in Manila)
#CHANGE THIS BACK
merged_amenities_points_gdf = gpd.read_file('./City Data/Manila_point.geojson')
merged_amenities_polygons_gdf= gpd.read_file('././City Data/Manila_polygon.geojson')

merged_amenities_polygons_gdf['amenity_points'] = None

In [54]:
# Getting total area to be used for the area connection type
merged_amenities_polygons_gdf['area'] = degrees_to_meters(merged_amenities_polygons_gdf['geometry'].area)
manila_area_sum = merged_amenities_polygons_gdf['area'].sum()

manila_area_sum

/var/folders/q0/zbq8f0v11vb5hv6jh21spfwm0000gn/T/ipykernel_63817/2666581131.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged_amenities_polygons_gdf['area'] = degrees_to_meters(merged_amenities_polygons_gdf['geometry'].area)


135.26544097431915

In [55]:
# Create spatial index for points
idx = rtree_index.Index()
for j, point in merged_amenities_points_gdf.iterrows():
    idx.insert(j, point['geometry'].bounds)

# Iterate over polygons
for i, polygon in merged_amenities_polygons_gdf.iterrows():
    points_within_polygon = []
    
    # Iterate over points within the bounding box of the polygon
    for j in idx.intersection(polygon['geometry'].bounds):
        point = merged_amenities_points_gdf.loc[j]
        if polygon['geometry'].intersects(point['geometry']):
            points_within_polygon.append(j)
    
    merged_amenities_polygons_gdf.at[i, 'amenity_points'] = points_within_polygon

#### Importing Manila Zone Network Data

In [56]:
# ------- LEVEL 1 
# IMPORT INITIAL NETWORK
initial_network_Manila = import_networks(f"{Manila_pikl_filepath}Manila_Initial_Network.pkl")

# IMPORT FILTERED NETWORK
filtered_manila_amenities_network = import_networks(f"{Manila_pikl_filepath}Manila_Filtered_Network.pkl")

# IMPORT COMBINED AMENITIES NETWORK
combined_graph_Manila = import_networks(f"{Manila_pikl_filepath}Manila_Combined_Amenities_Network.pkl")

# ------- LEVEL 2
# IMPORT POPULATION GRAPH
pop_graph = import_networks(f"{Manila_pikl_filepath}Manila_Population_Graph.pkl")

# IMPORT ZONE NETWORK
graph_networks_of_polygons_Manila = import_networks(f"{Manila_pikl_filepath}Manila_Zone_Network.pkl")
networks_map_Manila = plot_connected_zones_network_on_map(graph_networks_of_polygons_Manila, initial_location=[0, 0], zoom_start=100)

add_points_to_graph(initial_network_Manila, graph_networks_of_polygons_Manila) # Add all points

In [57]:
# Importing Stop list and Graph (Only to test the same stops)
list_of_stops_Manila = import_networks(f"{Manila_pikl_filepath}stop_list_Manila.pkl")
graph_of_stops_Manila = import_networks(f"{Manila_pikl_filepath}stop_graph_Manila.pkl")

# Visualize the stops
stops_map = plot_stops_on_map(list_of_stops_Manila)
stops_map.save(f"{Manila_map_filepath}stops_map.html") # Save the map to an HTML file

#### Stop Placement (Only to generate new stops)

In [ ]:
# LEVEL 3 - CREATING STOPS TO BE PLACED ON ZONES
graph_of_stops_Manila = copy.deepcopy(CITY_GRAPH)
list_of_stops_Manila = []
place_stops_on_roads(graph_networks_of_polygons_Manila, graph_of_stops_Manila, list_of_stops_Manila) # Adds stops graph_of_stops, and gets the relevant edges
if graph_of_stops_Manila.nodes(len(list_of_stops_Manila)) == {}:
    graph_of_stops_Manila.remove_node(len(list_of_stops_Manila))

# Visualize the stops
stops_map = plot_stops_on_map(list_of_stops_Manila, initial_location=[0, 0], zoom_start=100)
stops_map.save(f"{Manila_map_filepath}stops_map.html") # Save the map to an HTML file

In [ ]:
#Export stops to pickle
file_path = f'{Manila_pikl_filepath}stop_list_Manila.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(list_of_stops_Manila, f)
    
file_path = f'{Manila_pikl_filepath}stop_graph_Manila.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(graph_of_stops_Manila, f)

#### Stop Connection (Only to generate new routes)

In [60]:
# LEVEL 4 - CONNECTING STOPS INTO A NETWORK

# Configuration
set_walk_distance = WALKING_DISTANCES[0]
num_of_networks = 1
conn_type = CONNECTION_TYPES[0]
max_stops = 20
max_routes = 1 # temporary, should be 30
map_html_location = f"Generated Route Networks HTML/Manila/{conn_type}/"
        
# Generate route network
list_of_networks_Manila = []

print(f"CHOSEN CONNECTION TYPE: {conn_type}")
current_network_count = 0
for _ in range(num_of_networks):
    
    print(f"NETWORK {current_network_count}")
    if conn_type == "Mixed":
        temp_conn_type = random.choice(CONNECTION_TYPES[:-1])
        temp_walk_type = random.choice(WALKING_DISTANCES)
        print(f"NETWORK CONNECTION TYPE: {temp_conn_type}")
        route_network, route_graph = generate_route_network(list_of_stops_Manila, set_walk_distance, max_stops, max_routes, graph_of_stops_Manila, manila_area_sum, temp_conn_type) # Default max walking distance is 300m
        used_stops = add_stops_to_list(route_network)
        new_network = networkObj(route_network, used_stops, route_graph, temp_conn_type, temp_walk_type)
    else:
        route_network, route_graph = generate_route_network(list_of_stops_Manila, set_walk_distance, max_stops, max_routes, graph_of_stops_Manila, manila_area_sum, conn_type) # Default max walking distance is 300m
        used_stops = add_stops_to_list(route_network)
        new_network = networkObj(route_network, used_stops, route_graph, conn_type, set_walk_distance)
    
    # ERROR CHECKS----------
    print("Checking for graph and route consistency...", flush=True)
    check_graph_with_route(new_network)
    print("Checking for graph and list of stops consistency...", flush=True)
    check_graph_with_stops(new_network)
    print("Checking if order of routes is correct...", flush=True)
    check_order_route(new_network.routes)
    print("Checking for list of stops and route consistency...", flush=True)
    check_stops_routes(new_network)
    print("Checking for duplicates in routes...")
    check_duplicate_routes(new_network)
    
    
    print()
    list_of_networks_Manila.append(new_network) # Append to list of networks
    current_network_count += 1

#Export networks and graphs using pickl
if conn_type == 'Mixed':
    export_networks(list_of_networks_Manila, f"{Manila_pikl_filepath}Manila_Route_networks_{conn_type}.pkl")
else:
    export_networks(list_of_networks_Manila, f"{Manila_pikl_filepath}Manila_Route_networks_{conn_type}_{set_walk_distance}.pkl")

CHOSEN CONNECTION TYPE: Default
NETWORK 0
# OF CONNECTIONS AND TOTAL DISTANCE: 8 - 14.689748762736485
Checking for graph and route consistency...
Checking for graph and list of stops consistency...
Checking if order of routes is correct...
Checking for list of stops and route consistency...
Checking for duplicates in routes...



In [61]:
# Creating Maps for visualization

i = 1
for route_network in list_of_networks_Manila:
    map_center = (14.599512, 120.984222)
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Plotting in the Map
    add_markers(route_network.stops, route_network.graph)
        
    for route in route_network.routes:
        for connection in route.route:
            ox.plot_route_folium(graph_of_stops_Manila, connection, route_map=m, tiles='openstreetmap', route_color="green")

    m.save(f"{map_html_location}Route Map-{i}.html")
    i += 1

/var/folders/q0/zbq8f0v11vb5hv6jh21spfwm0000gn/T/ipykernel_63817/2616100919.py:13: FutureWarning: The `folium` module has been deprecated and will be removed in the v2.0.0 release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  ox.plot_route_folium(graph_of_stops_Manila, connection, route_map=m, tiles='openstreetmap', route_color="green")
/var/folders/q0/zbq8f0v11vb5hv6jh21spfwm0000gn/T/ipykernel_63817/2616100919.py:13: FutureWarning: The `folium` module has been deprecated and will be removed in the v2.0.0 release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples galler